In [1]:
# !pip install MTCFeatures

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import MTCFeatures
from MTCFeatures import MTCFeatureLoader
import pandas as pd

In [3]:
MTCFeatures.downloadData(dest='user')

[██████████████████████████████████████████████████]MTC-ANN-2.0.1_sequences-1.1.jsonl.gz?download=1 -> /home/gaspar/.local/share/MTCFeatures/MTC-ANN-2.0.1_sequences-1.1.jsonl.gz
[██████████████████████████████████████████████████]MTC-FS-INST-2.0_sequences-1.1.jsonl.gz?download=1 -> /home/gaspar/.local/share/MTCFeatures/MTC-FS-INST-2.0_sequences-1.1.jsonl.gz
[██████████████████████████████████████████████████]essen_sequences-1.1.jsonl.gz?download=1 -> /home/gaspar/.local/share/MTCFeatures/essen_sequences-1.1.jsonl.gz


In [4]:
fl = MTCFeatureLoader('MTC-FS-INST-2.0')
seqs = fl.sequences()

In [5]:
%%time
phrase_data = []
for ii, x in enumerate(seqs):
    phrase_data.append({
        'id': x['id'],
        **x['features']
    })

CPU times: user 13.9 s, sys: 1.49 s, total: 15.4 s
Wall time: 15.4 s


In [6]:
df_phrase = pd.DataFrame(phrase_data)
df_phrase

,id,scaledegree,scaledegreespecifier,tonic,mode,metriccontour,imaweight,pitch40,midipitch,diatonicpitch,...,durationcontour,IOR_frac,lyrics,noncontentword,wordend,phoneme,rhymes,rhymescontentwords,wordstress,melismastate
0,NLB125814_01,"[3, 1, 6, 5, 1, 1, 2, 1, 2, 3, 6, 5, 5, 4, 5, ...","[M, P, M, P, P, P, M, P, M, M, M, P, P, A, P, ...","[C, C, C, C, C, C, C, C, C, C, C, C, C, C, C, ...","[major, major, major, major, major, major, maj...","[None, -, -, +, -, +, +, -, -, +, -, +, +, -, ...","[0.798013, 0.663907, 0.150662, 0.945364, 0.013...","[135, 123, 112, 106, 123, 123, 129, 123, 129, ...","[64, 60, 57, 55, 60, 60, 62, 60, 62, 64, 57, 5...","[30, 28, 26, 25, 28, 28, 29, 28, 29, 30, 26, 2...",...,"[None, -, =, +, -, +, =, -, =, =, =, +, =, -, ...","[None, 1/2, 1, 3/2, 1/3, 4, 1, 1/2, 1, 1, 1, 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NLB125815_01,"[5, 5, 6, 5, 4, 3, 1, 3, 3, 3, 3, 6, 5, 5, 6, ...","[P, A, M, P, P, M, P, M, M, M, M, M, P, P, M, ...","[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...","[major, major, major, major, major, major, maj...","[None, -, +, -, +, -, +, -, -, +, -, +, +, -, ...","[0.236646, 0.015528, 0.913043, 0.204348, 0.392...","[163, 164, 169, 163, 157, 152, 140, 152, 152, ...","[72, 73, 74, 72, 70, 69, 65, 69, 69, 69, 69, 7...","[32, 32, 33, 32, 31, 30, 28, 30, 30, 30, 30, 3...",...,"[None, -, +, =, =, =, -, -, =, +, =, +, -, -, ...","[None, 1/3, 4, 1, 1, 1, 1/2, 1/2, 1, 2, 1, 4, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NLB125817_01,"[5, 5, 5, 1, 5, 3, 2, 5, 2, 3, 2, 1, 2, 3, 4, ...","[P, P, P, P, P, M, M, P, M, M, M, P, M, M, P, ...","[C, C, C, C, C, C, C, C, C, C, C, C, C, C, C, ...","[major, major, major, major, major, major, maj...","[None, +, -, +, -, +, -, +, -, +, -, +, -, +, ...","[0.246883, 0.306733, 0.169576, 0.780549, 0.349...","[146, 146, 146, 163, 146, 175, 169, 146, 169, ...","[67, 67, 67, 72, 67, 76, 74, 67, 74, 76, 74, 7...","[32, 32, 32, 35, 32, 37, 36, 32, 36, 37, 36, 3...",...,"[None, =, =, +, -, +, -, +, -, +, -, =, =, =, ...","[None, 1, 1, 3, 1/3, 3, 1/3, 6, 1/3, 3/2, 1/3,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NLB125818_01,"[1, 1, 1, 1, 3, 2, 1, 7, 2, 2, 5, 4, 3, 4, 3, ...","[P, P, P, P, m, M, P, m, M, M, P, P, m, P, m, ...","[C, C, C, C, C, C, C, C, C, C, C, C, C, C, C, ...","[minor, minor, minor, minor, minor, minor, min...","[None, +, -, +, +, -, -, +, -, -, +, -, +, -, ...","[0.080378, 0.617021, 0.202128, 0.361702, 0.320...","[163, 163, 163, 163, 174, 169, 163, 157, 169, ...","[72, 72, 72, 72, 75, 74, 72, 70, 74, 74, 79, 7...","[35, 35, 35, 35, 37, 36, 35, 34, 36, 36, 39, 3...",...,"[None, =, =, +, =, -, =, +, -, =, +, -, +, -, ...","[None, 1, 1, 2, 1, 1/2, 1, 2, 1/2, 1, 3, 1/3, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NLB125822_01,"[5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, ...","[P, P, P, P, P, P, P, M, M, M, M, M, M, M, M, ...","[F, F, F, F, F, F, F, F, F, F, F, F, F, F, F, ...","[major, major, major, major, major, major, maj...","[None, -, -, +, -, -, +, -, +, +, -, -, +, -, ...","[0.971634, 0.069592, 0.005484, 0.783094, 0.071...","[163, 163, 163, 163, 163, 163, 163, 152, 152, ...","[72, 72, 72, 72, 72, 72, 72, 69, 69, 69, 69, 6...","[32, 32, 32, 32, 32, 32, 32, 30, 30, 30, 30, 3...",...,"[None, -, =, +, -, =, +, =, +, -, -, =, +, -, ...","[None, 1/2, 1, 2, 1/2, 1, 2, 1, 2, 1/2, 1/2, 1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18104,NLB179585_01,"[5, 3, 5, 3, 1, 1, 1, 2, 4, 7, 2, 1, 7, 1, 2, ...","[P, M, P, M, P, P, P, M, P, M, M, P, M, P, M, ...","[B-, B-, B-, B-, B-, B-, B-, B-, B-, B-, B-, B...","[major, major, major, major, major, major, maj...","[None, -, +, -, +, -, +, +, -, +, -, +, -, +, ...","[0.913915, 0.194076, 0.253625, 0.138537, 0.914...","[180, 169, 180, 169, 157, 157, 157, 163, 174, ...","[77, 74, 77, 74, 70, 70, 70, 72, 75, 69, 72, 7...","[32, 30, 32, 30, 28, 28, 28, 29, 31, 27, 29, 2...",...,"[None, =, =, =, =, =, +, -, =, =, =, -, =, =, ...","[None, 1, 1, 1, 1, 1, 2, 1/2, 1,

In [17]:

import numpy as np

In [27]:
# Chaque ligne du set représente une mélodie, représentée par des features.

print(np.array(df_phrase['pitch'][0]).shape) # pitch représente la hauteur des notes
print(np.array(df_phrase['duration'][0]).shape) # duration représente la durée des notes
print(np.array(df_phrase['phrase_end'][0]).shape) # phrase_end dit si la note est la dernière de la phrase

print(df_phrase['pitch'][0])
print(df_phrase['duration'][0])
print(df_phrase['phrase_end'][0])
# On peut donc voir que chacun de ses 3 features est une liste de la même longueur que le nombre de notes dans la mélodie.

# combien de fin de phrase dans le premier exemple ?

print(np.sum(df_phrase['phrase_end'][0])) # 4

# On pense découpere les séquences en sous séquences par notes avec un nombre fixe de notes par sous séquence. On pourra tester de diviser les séquences en sous séquences de nombre 
# fixe de note pour toutes les séquences ou bien de diviser les séquences par un nombre fixe pour toutes les séquences.

print(df_phrase['beat'][0]) # beat représente le temps de la note dans la mesure

# On peut aussi faire des sous séquences basées sur les mesures pour éviter de couper une mesure en deux.

print(df_phrase['beat_fraction_str'][0]) # beat_fraction_str représente le temps de la note dans la mesure sous forme de fraction

(44,)
(44,)
(44,)
['E4', 'C4', 'A3', 'G3', 'C4', 'C4', 'D4', 'C4', 'D4', 'E4', 'A3', 'G3', 'G3', 'F#3', 'G3', 'A3', 'F4', 'E4', 'D4', 'E4', 'D4', 'G3', 'D4', 'E4', 'D4', 'C4', 'A3', 'G3', 'E4', 'F4', 'E4', 'A3', 'E4', 'D4', 'G3', 'A3', 'G3', 'G3', 'F4', 'E4', 'D4', 'A3', 'B3', 'C4']
[1.0, 0.5, 0.5, 0.75, 0.25, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.5]
[False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True]
4
[1.0, 2.0, 2.5, 1.0, 1.75, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5, 1.0, 1.5, 2.0, 1.0, 2.0, 2.5,